## Mountain Car Environment

Kenechi Franklin Dukor

Special thanks to Facebook-Pytorch and Udacity

#### Lets import the neccessary packages

In [2]:
import sys
import gym
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt

# Set plotting options
%matplotlib inline
plt.style.use('ggplot')
np.set_printoptions(precision=3, linewidth=120)

Lets create an instance of the [MountainCar](https://gym.openai.com/) environment provided by `openai gym`

In [3]:
# Create an environment and set random seed
env = gym.make('MountainCar-v0')
env.seed(505);

Run the next code cell to watch a random agent.

In [4]:
state = env.reset()
score = 0
for t in range(200):
    action = env.action_space.sample()
    env.render()
    state, reward, done, info = env.step(action)
    score += reward
    if done:
        break 
print('Final score:', score)
env.close()

Final score: -200.0


Next, lets explore the state and action spaces, as well as sample them.

In [5]:
# Explore state (observation) space
print("State space:", env.observation_space)
print("- low:", env.observation_space.low)
print("- high:", env.observation_space.high)

State space: Box(2,)
- low: [-1.2  -0.07]
- high: [0.6  0.07]


In [6]:
# Generate some samples from the state space 
print("State space samples:")
print(np.array([env.observation_space.sample() for i in range(10)]))

State space samples:
[[ 0.572  0.012]
 [ 0.578  0.038]
 [-0.507 -0.04 ]
 [-0.095  0.042]
 [-0.78   0.048]
 [-0.133 -0.003]
 [-1.153  0.041]
 [ 0.532  0.025]
 [-0.569 -0.018]
 [-0.818  0.019]]


Notice that the state space is continous

In [7]:
# Explore the action space
print("Action space:", env.action_space)

# Generate some samples from the action space
print("Action space samples:")
print(np.array([env.action_space.sample() for i in range(10)]))

Action space: Discrete(3)
Action space samples:
[1 1 2 1 1 2 0 1 0 2]
